In [3]:
import sys; sys.path.append('../3rdparty/ElasticRods/python')
import elastic_rods, elastic_knots
import numpy as np, matplotlib.pyplot as plt, time, io, os
from scipy.spatial.distance import cdist

from helpers import *
from parametric_curves import *
import py_newton_optimizer

from linkage_vis import LinkageViewer as Viewer, CenterlineViewer
from tri_mesh_viewer import PointCloudViewer, PointCloudMesh

%load_ext autoreload
%autoreload 2

import parallelism
parallelism.set_max_num_tbb_threads(1)

ModuleNotFoundError: No module named 'elastic_rods'

In [39]:
#Returns the index of list b in a
def findSequence(a,b):
    for i in range(len(a)):
        if np.array_equal(a[i:i+len(b)],np.array(b)):
              return (i, i+len(b))
    

In [40]:
def findNearestVertex(A,B):
    distances = cdist(A, B)
    min_idx = np.unravel_index(np.argmin(distances), distances.shape)
    return min_idx[0], min_idx[1]

In [41]:
def interpolateEdge(a,b):
    return

In [42]:
def combineKnots(A,B):
    B=B+5
    minA, minB = findNearestVertex(A,B)
    A_head = A[:minA]
    A_tail = A[minA+1:]
    B_reordered = np.concatenate([B[minB:], B[:minB]], axis=0)
    combined = np.concatenate([A_head, B_reordered, A_tail], axis=0)
    
    return combined, B_reordered

    
    

In [43]:
def doubleKnot(A,B):
    B = B+0.5
    combined = np.concatenate([A,B], axis=0)
    
    return combined

In [44]:
# Load the centerline from file...
file = '../data/L400-r0.2-UpTo9Crossings/4_1/0001.obj'
file = '../data/CubicLatticeKnots/L100/4_1.txt'
file2 = '../data/CubicLatticeKnots/L300/4_1.txt'
rod_radius = 0.2
material = elastic_rods.RodMaterial('ellipse', 2000, 0.3, [rod_radius, rod_radius])
#combined = combineKnots1(read_nodes_from_file(file), read_nodes_from_file(file))
A = read_nodes_from_file(file2)
B = read_nodes_from_file(file)
circle = doubleKnot(B, np.flip(B,axis=0))
combined , B_reorderd= combineKnots(np.flip(A,axis=0), circle)
B_ind = findSequence(combined,B_reorderd)
print(B_ind)
#combined = combineKnots(read_nodes_from_file(file), np.flip(read_nodes_from_file(file),axis=0))
pr = define_periodic_rod(combined, material)
#pr = define_periodic_rod(circle, material)
rod_list = elastic_knots.PeriodicRodList([pr])
rod_list.energy()

(43, 243)


1727.608437923668

In [45]:
view = Viewer(rod_list, width=1024, height=1000)
view.show()

Renderer(camera=PerspectiveCamera(aspect=1.024, children=(PointLight(color='#999999', position=(0.0, 0.0, 5.0)…

In [46]:
def callback(problem, iteration):
    if iteration % 5 == 0:
        view.update()
        
optimizerOptions = py_newton_optimizer.NewtonOptimizerOptions()
optimizerOptions.niter = 1000
optimizerOptions.gradTol = 1e-8
hessianShift = 1e-4 * compute_min_eigenval_straight_rod(pr)

problemOptions = elastic_knots.ContactProblemOptions()
problemOptions.contactStiffness = 1e3
problemOptions.dHat = 2*rod_radius

fixedVars = range(B_ind[0]*3, int((B_ind[1]*3)/2))   
#fixedVars = []

In [47]:
# Optimize
def optimize():
    report = elastic_knots.compute_equilibrium(
        rod_list, problemOptions, optimizerOptions, 
        fixedVars=fixedVars,
        externalForces=np.zeros(rod_list.numDoF()),
        softConstraints=[],
        callback=callback,
        hessianShift=hessianShift
        )
    view.update()

In [ ]:
optimize()
optimize()
optimize()
optimize()
optimize()
optimize()
optimize()
optimize()

0	1727.61	364.165	364.165	1	1
1	1666.09	303.256	303.256	1	1
2	1641.2	279.109	279.109	1	1
3	1602.15	244.753	244.753	1	1
4	1585.32	231.062	231.062	1	1
5	1556.86	211.089	211.089	1	1
6	1543.99	203.018	203.018	1	1
7	1521.04	190.817	190.817	1	1
8	1510.33	185.763	185.763	1	1
9	1490.58	177.771	177.771	1	1
10	1481.19	174.354	174.354	1	1
11	1463.49	168.692	168.692	1	1
12	1454.98	166.191	166.191	1	1
13	1438.74	161.871	161.871	1	1
14	1430.86	159.908	159.908	1	1
15	1415.73	156.399	156.399	1	1
16	1408.36	154.769	154.769	1	1
17	1394.13	151.781	151.781	1	1
18	1387.17	150.369	150.369	1	1
19	1373.7	147.733	147.733	1	1
20	1367.1	146.472	146.472	1	1
21	1354.29	144.087	144.087	1	1
22	1348.01	142.937	142.937	1	1
23	1335.79	140.74	140.74	1	1
24	1329.8	139.674	139.674	1	1
25	1318.12	137.625	137.625	1	1
26	1312.38	136.626	136.626	1	1
27	1301.19	134.697	134.697	1	1
28	1295.7	133.753	133.753	1	1
29	1284.96	131.925	131.925	1	1
30	1279.69	131.028	131.028	1	1
31	1269.38	129.286	129.286	1	1
32	1264.31	128.429	128.42

In [20]:
color_rods(view, rod_list)   # dark blue and dark red represent the first and last edges of each rod, respectively